In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
# Load and preprocess the dataset
df = pd.read_csv('song_dataset.csv')
df.drop_duplicates(inplace=True)  # Remove duplicate rows
df.dropna(inplace=True)  # Remove rows with missing values

# Create a user-song matrix for the recommendation system
user_song_matrix = df.pivot_table(index='user', columns='song', values='play_count').fillna(0)

# Initialize and train the NearestNeighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(user_song_matrix)

# Define the song recommendation function
def recommend_songs(user_id, data, model, original_df, listened_song, n_recommendations=5):
    if user_id not in data.index:
        return ["User ID not found."]
    if listened_song not in original_df['title'].values:
        return ["Song not found."]
    
    user_index = data.index.tolist().index(user_id)
    distances, indices = model.kneighbors(data.iloc[user_index, :].values.reshape(1, -1), n_neighbors=n_recommendations+10)
    
    recommendations = []
    for i in range(1, len(distances.flatten())):
        song_id = data.columns[indices.flatten()[i]]
        if len(recommendations) < n_recommendations:
            song_info = original_df[original_df['song'] == song_id].iloc[0]
            if song_info['title'] != listened_song:
                recommendations.append({
                    'title': song_info['title'],
                    'release': song_info['release'],
                    'artist_name': song_info['artist_name'],
                    'year': song_info['year']
                })
    return recommendations

# Create interactive widgets
user_id_input = widgets.Text(description='User ID:')
listened_song_input = widgets.Text(description='Listened Song:')
recommend_button = widgets.Button(description='Recommend')
output = widgets.Output()

# Function to handle the Recommend button click
def on_recommend_button_clicked(b):
    with output:
        clear_output()
        recommendations = recommend_songs(user_id_input.value, user_song_matrix, model_knn, df, listened_song_input.value)

        if recommendations and isinstance(recommendations[0], dict):
            for rec in recommendations:
                print(f"Title: {rec['title']}, Artist: {rec['artist_name']}, Release: {rec['release']}, Year: {rec['year']}")
        else:
            for rec in recommendations:
                print(rec)


# Link the button click event to the handler function
recommend_button.on_click(on_recommend_button_clicked)

# Display the widgets
display(user_id_input, listened_song_input, recommend_button, output)


Text(value='', description='User ID:')

Text(value='', description='Listened Song:')

Button(description='Recommend', style=ButtonStyle())

Output()

In [2]:
# Test
user_id_input.value = 'b80344d063b5ccb3212f76538f3d9e43d87dca9e'
listened_song_input.value = 'Stronger'


on_recommend_button_clicked(None)


display(user_id_input, listened_song_input, recommend_button, output)


Text(value='b80344d063b5ccb3212f76538f3d9e43d87dca9e', description='User ID:')

Text(value='Stronger', description='Listened Song:')

Button(description='Recommend', style=ButtonStyle())

Output()